In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib.parse
from tqdm.notebook import tqdm

In [4]:
def parse_entry_line(line):
    cells = line.find_all('td')
    if len(cells) < 4:
        return False
    full_name = cells[1].a.text
    club = cells[2].text
    full_name = full_name.replace("ё", "е").replace("Ё", "Е")
    club = club.replace("ё", "е").replace("Ё", "Е")
    full_name = [x.strip() for x in full_name.split()]
    if len(full_name) == 2:
        first_name = full_name[0]
        middle_name = ""
        last_name = full_name[1]
    elif len(full_name) == 3:
        first_name = full_name[0]
        middle_name = full_name[1]
        last_name = full_name[2]
    else:
        return False
    return {
        'firstname': first_name,
        'middlename': middle_name,
        'lastname': last_name,
        'club': club,
    }

In [5]:
def parse_event(event_link, data, judges):
    soup = BeautifulSoup(requests.get(event_link).text, 'html.parser')
    title = soup.find("h1", "entry-title").text
    date = soup.find("div", "competition-date").p.text.split('-')[0].strip()
    place = soup.find("div", "competition-place").p.span.text.strip()
    online = soup.find("div", "competition-file")
    if online is not None:
        online_link = online.a.get('href')
        online_link = online_link if online_link[-1] == '/' else online_link + '/'
        soup = BeautifulSoup(requests.get(online_link).text, 'html.parser')
        lines = soup.find_all('tr')

        for line in lines:
            columns = line.find_all('td')
            if (len(columns) < 5):
                continue
            if (columns[0].text != ""):
                category = columns[0].text                
                soup = BeautifulSoup(requests.get(urllib.parse.urljoin(online_link, columns[2].a.get("href"))).text, 'html.parser')
                entries = soup.find_all('tr')
                persons = []
                for entry in entries:
                    pdata = parse_entry_line(entry) 
                    if pdata:
                        persons.append(pdata)
            if (columns[1].text != ""):
                segment = columns[1].text
                for person in persons:
                    data['date'].append(date)
                    data['place'].append(place)
                    data['category'].append(category)
                    data['segment'].append(segment)
                    data['firstname'].append(person['firstname'])
                    data['middlename'].append(person['middlename'])
                    data['lastname'].append(person['lastname'])
                    data['club'].append(person['club'])

In [6]:
data = {
    'date': [],
    'place': [], 
    'category': [],
    'segment': [],
    'firstname': [],
    'middlename': [],
    'lastname': [],
    'club': [],
}
judges = {
    'date': [],
    'place': [], 
    'function': [],
    'name': [],
}
parse_event("http://ffkkmo.ru/competition/%d1%81%d0%bf%d0%be%d1%80%d1%82%d0%b8%d0%b2%d0%bd%d1%8b%d0%b5-%d1%81%d0%be%d1%80%d0%b5%d0%b2%d0%bd%d0%be%d0%b2%d0%b0%d0%bd%d0%b8%d1%8f-%d0%bc%d0%be%d1%81%d0%ba%d0%be%d0%b2%d1%81%d0%ba%d0%be%d0%b9-121/",
           data, judges)

In [7]:
pd.DataFrame.from_dict(data)

,date,place,category,segment,firstname,middlename,lastname,club
0,02.10.2025,"г. Ногинск, ул. Климова, д. 48, корпус А, ФОК ...","Юный фигуриcт, дeвoчки",Произвольная программа,Варвара,Ильинична,КЛЕБАНЮК,"г. Ногинск, МБУ ДО БГО ""СШОР"""
1,02.10.2025,"г. Ногинск, ул. Климова, д. 48, корпус А, ФОК ...","3-й юнoшecкий разряд, дeвoчки",Произвольная программа,Виталия,Михайловна,АНДРЕЕВА,"г. Ногинск, МБУ ДО БГО ""СШОР"""
2,02.10.2025,"г. Ногинск, ул. Климова, д. 48, корпус А, ФОК ...","3-й юнoшecкий разряд, дeвoчки",Произвольная программа,Кристина,Александровна,АНТОНОВА,"г. о. Орехово-Зуевский, МУ ДО СШ ""Спартак-Орех..."
3,02.10.2025,"г. Ногинск, ул. Климова, д. 48, корпус А, ФОК ...","3-й юнoшecкий разряд, дeвoчки",Произвольная программа,Полина,Дмитриевна,БАКЛАГИНА,"г. о. Орехово-Зуевский, МУ ДО СШ ""Спартак-Орех..."
4,02.10.2025,"г. Ногинск, ул. Климова, д. 48, корпус А, ФОК ...","3-й юнoшecкий разряд, дeвoчки",Произвольная программа,София,Антоновна,БОГДАНОВА,"г. о. Жуковский, МБУ ДО ""СШ - Центр спорта ""Ме..."
5,02.10.2025,"г. Ногинск, ул. Климова, д. 48, корпус А, ФОК ...","3-й юнoшecкий разряд, дeвoчки",Произвольная программа,Софья,Вячеславовна,ВАСИЛКО,"г. о. Орехово-Зуевский, МУ ДО СШ ""Спартак-Орех..."
6,02.10.2025,"г. Ногинск, ул. Климова, д. 48, корпус А, ФОК ...","3-й юнoшecкий разряд, дeвoчки",Произвольная программа,Александра,Дмитриевна,ДОЛОВА,"г. о. Раменское, МБУ ДО ""СШ ""Раменское"""
7,02.10.2025,"г. Ногинск, ул. Климова, д. 48, корпус А, ФОК ...","3-й юнoшecкий разряд, дeвoчки",Произвольная программа,Василиса,Сергеевна,КИЗЫМА,"г. Ногинск, МБУ ДО БГО ""СШОР"""
8,02.10.2025,"г. Ногинск, ул. Климова, д. 48, корпус А, ФОК ...","3-й юнoшecкий разряд, дeвoчки",Произвольная программа,Алиса,Александровна,НАУМОВА,"г. о. Мытищи, СК ""Ледовая феерия"", ИП Киселева..."
9,02.10.2025,"г. Ногинск, ул. Климова, д. 48, корпус А, ФОК ...","3-й юнoшecкий разряд, дeвoчки",Произвольная программа,Маргарита,Александровна,РУБЦОВА,"г. о. Балашиха, МАУ ДО ""СШОР им. Ю. Е. Ляпкина"""
